Para simular a situação, criei um arquivo .CSV e converti para .parquet

In [ ]:
import pandas as pd
from datetime import datetime

def file_to_data_frame_to_parquet(local_file: str, parquet_file: str) -> None:
    df = pd.read_csv(local_file)
    df.to_parquet(parquet_file)

if __name__ == "__main__":
    local_csv_file = "sensor.csv"
    t1 = datetime.now()
    file_to_data_frame_to_parquet(local_csv_file, "sensor.parquet")
    t2 = datetime.now()
    took = t2 - t1
    print(f"it took {took} seconds to write csv to parquet.")

it took 0:00:00.217996 seconds to write csv to parquet.


Converte texto para timestamp

In [ ]:
odbc_df = pd.read_parquet("sensor.parquet")
odbc_df['timestamp'] = pd.to_datetime(odbc_df['timestamp'])
odbc_df.head()

,sensor_name,timestamp,value
0,A801,2023-01-31 12:01:00,14.9
1,A801,2023-01-31 12:02:00,14.7
2,A801,2023-01-31 12:03:00,14.7
3,A801,2023-01-31 12:04:00,14.5
4,A801,2023-01-31 12:05:00,14.5


efectua uma primeira carga de dados da origem ODBC para uma base temporária.


In [ ]:
temp_df = odbc_df
temp_df.head()

,sensor_name,timestamp,value
0,A801,2023-01-31 12:01:00,14.9
1,A801,2023-01-31 12:02:00,14.7
2,A801,2023-01-31 12:03:00,14.7
3,A801,2023-01-31 12:04:00,14.5
4,A801,2023-01-31 12:05:00,14.5


Efectua a leitura da origem ODBC, filtrando somente a última hora

In [ ]:
hour_df = odbc_df[odbc_df['timestamp'] > odbc_df['timestamp'].max() - pd.Timedelta(hours=1)]
hour_df.head()

,sensor_name,timestamp,value
0,A801,2023-01-31 12:01:00,14.9
1,A801,2023-01-31 12:02:00,14.7
2,A801,2023-01-31 12:03:00,14.7
3,A801,2023-01-31 12:04:00,14.5
4,A801,2023-01-31 12:05:00,14.5


Carrega a tabela agregada, dos dados da temporária com a extração de 1h

In [ ]:
aggregated_db = pd.concat([temp_df, hour_df])
aggregated_db.head()

,sensor_name,timestamp,value
0,A801,2023-01-31 12:01:00,14.9
1,A801,2023-01-31 12:02:00,14.7
2,A801,2023-01-31 12:03:00,14.7
3,A801,2023-01-31 12:04:00,14.5
4,A801,2023-01-31 12:05:00,14.5
